In [2]:
from core import *

In [2]:
# load dataset
usedDatas = [[('cloud', 'psf', 'qvapor', 'rain', 'sst', 'wind', 'winu', 'wn10'),],[('psf','qvapor','rain','sst','wind','winu','wn10'),],[('cloud','psf','qvapor','rain','sst','wind','winu'),],[('cloud','psf','qvapor','rain','sst','wind','wn10'),],[('psf','qvapor','rain','sst','wind','winu'),]]
dims=([784,],[784,],[784,],[784,],[784,])
recurrentLists = (1,2,3,4,5,6)
WLSTATION = 'katulampa'
DATASCOPE = 'prediction'

# SLEEP #
#time.sleep(14400) # 4 hours waiting for manggarai RNN to be finished
#########

adte, adta = getAvailableSlicedData(dataScope=DATASCOPE, wlstation=WLSTATION)
adta = np.array(adta).astype('float32')
# normalizing input data
norm, minStat, maxStat = normalizingLabels(adta)

dataLabel = []
# create data Label for each recurrent number
for recurrentList in recurrentLists:
    recurrentIndexList, availableRecurrentDate, availableRecurrentLabel = generateRNNInput(adte, norm, recurrentCount=recurrentList)
    dataLabel.append(availableRecurrentLabel)
    
# DEFINE THIS FIRST #
usedDatas = usedDatas[0]
dims = dims[0]
# ################# #

himawariData={}
for i in range(len(usedDatas)):
    usedData = usedDatas[i]
    dim = dims[i]
    # define the length of data
    dataLength = len(usedData)
    # read stored data
    if dataLength == 1:
        fileName = '{}{}'.format(usedData[0], dim)
    elif dataLength == 2:
        fileName = '{}{}{}'.format(usedData[0], usedData[1], dim)
    elif dataLength == 3:
        fileName = '{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], dim)
    elif dataLength == 4:
        fileName = '{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], dim)
    elif dataLength == 5:
        fileName = '{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], dim)
    elif dataLength == 6:
        fileName = '{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], dim)
    elif dataLength == 7:
        fileName = '{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], dim)
    elif dataLength == 8:
        fileName = '{}{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], usedData[7], dim)
    print(fileName)
    
    for j in range(len(recurrentLists)):
        fullFileName = '{}r{}f'.format(fileName, recurrentLists[j])
        fpath = 'dataset/{}RNN/{}.hdf5'.format(WLSTATION, fullFileName)
        with h5py.File(fpath,'r') as f:
            data = f['datas'][()]
            
        print(data.shape, len(dataLabel[j]))

        himawariData[fullFileName]={'filename':'{}'.format(fullFileName),
                                    'traintest': splitTrainTest(data, dataLabel[j], split=0.7, shuffle=True, randomSeed=10)}


cloudpsfqvaporrainsstwindwinuwn10784
(8177, 12544) 8177
(6400, 18816) 6400
(5105, 25088) 5105
(4137, 31360) 4137
(3403, 37632) 3403
(2815, 43904) 2815


In [3]:
# Error function
def rmse(yreal, ypred):
    return np.sqrt(np.mean(np.square(yreal-ypred)))

def nse(yreal, ypred):
    a = np.sum(np.square(ypred-yreal))
    b = np.sum(np.square(yreal-np.mean(yreal)))
    return 1-(a/b)

def nseModel(yreal, ypred):
    a = keras.backend.sum(keras.backend.square(ypred-yreal))
    b = keras.backend.sum(keras.backend.square(yreal-keras.backend.mean(yreal)))
    return a/b

def rmseModel(yreal, ypred):
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square(yreal - ypred)))

def rsquaredModel(yreal, ypred):
    #total_error = tf.math.reduce_sum(tf.math.square(tf.math.subtract(yreal, tf.math.reduce_mean(yreal))))
    #unexplained_error = tf.math.reduce_sum(tf.math.square(tf.math.subtract(yreal, ypred)))
    #R_squared = tf.math.subtract(1, tf.math.divide(unexplained_error, total_error))
    total_error = keras.backend.reduce_sum(keras.backend.square(keras.backend.subtract(yreal, keras.backend.reduce_mean(yreal))))
    unexplained_error = keras.backend.reduce_sum(keras.backend.square(keras.backend.subtract(yreal, ypred)))
    R_squared = keras.backend.subtract(1, keras.backend.divide(unexplained_error, total_error))
    return R_squared

def model6(flayer, slayer, epoch):
    tf.keras.backend.clear_session()
    initializer = tf.keras.initializers.GlorotNormal()
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(trainData.shape[1],)),
        tf.keras.layers.Dense(flayer, activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Dense(slayer, activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Dense(1)
    ])

    # compiling model
    opt = tf.keras.optimizers.Adam()
    model.compile(optimizer='adam',
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])

    # feed and train the model
    model.fit(trainData, trainLabel, epochs=epoch, verbose=0)
    return model

In [ ]:
# model variables
iteration = '66'

flayers = [4,8,12,16]
slayers = [1,2,3,4,5,6]
epochs = [50,100,250,500]

loops = 0
#hotStart = 202
for fullFileName in himawariData.keys():
    (trainData, trainLabel), (testData, testLabel) = himawariData[fullFileName]['traintest']
    for flayer in flayers:
        for slayer in slayers:
            for epoch in epochs:
                # hotstart
                #if loops < hotStart:
                    #loops+=1
                    #continue
                
                tick = datetime.datetime.now()
                model = model6(flayer, slayer, epoch)

                # evaluating model accuracy
                prediction_model = tf.keras.Sequential([model,
                                                        tf.keras.layers.ReLU()])
                testPredictions = prediction_model.predict(testData)
                trainPredictions = prediction_model.predict(trainData)

                # make predictions
                testPredictions = testPredictions*(maxStat-minStat)+minStat
                trainPredictions = trainPredictions*(maxStat-minStat)+minStat
                realTestLabel = testLabel*(maxStat-minStat)+minStat
                realTrainLabel = trainLabel*(maxStat-minStat)+minStat

                # Mean Squared Error : 
                mse = tf.keras.losses.MeanSquaredError()
                mseTestError = mse(realTestLabel, testPredictions).numpy()
                mseTrainError = mse(realTrainLabel, trainPredictions).numpy()

                # RMSE
                rmseTest = rmse(np.squeeze(testPredictions), realTestLabel)
                rmseTrain = rmse(np.squeeze(trainPredictions), realTrainLabel)

                # NSE
                nseTest = nse(realTestLabel, np.squeeze(testPredictions))
                nseTrain = nse(realTrainLabel, np.squeeze(trainPredictions))

                # R^2
                r2Test = r2_score(realTestLabel, testPredictions)
                r2Train = r2_score(realTrainLabel, trainPredictions)

                # save statistics to csv
                statistics = '{},{},{},{},{},{},{},{},{},{},{}\n'.format(flayer, slayer, epoch, mseTrainError, mseTestError, rmseTrain, rmseTest, r2Train, r2Test, nseTrain, nseTest)
                with open('models_statistics/{}_GS_{}.csv'.format(iteration, himawariData[fullFileName]['filename']), 'a') as stat:
                    stat.write(statistics)

                # save model to drive
                model.save('models/{}/{}_GS_{}_{}_{}.h5'.format(iteration, himawariData[fullFileName]['filename'], flayer, slayer, epoch))

                # loop identifier :
                tock = datetime.datetime.now()
                print('{} : {} - {} - {} - {} : time : {} - R^2 err : train[{}] test[{}]'.format(loops, himawariData[fullFileName]['filename'], flayer, slayer, epoch, tock-tick, r2Train, r2Test))
                loops+=1
      

0 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 1 - 50 : time : 0:01:26.354996 - R^2 err : train[-9.620391206555112e-06] test[-0.000761503432502364]
1 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 1 - 100 : time : 0:02:36.956003 - R^2 err : train[-0.0015801856646837642] test[-0.005212389741347412]
2 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 1 - 250 : time : 0:06:08.544998 - R^2 err : train[-0.000543855156434514] test[-4.2995684335744855e-05]
3 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 1 - 500 : time : 0:12:16.933003 - R^2 err : train[-4.688169223276972e-06] test[-0.0008159560319958015]
4 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 2 - 50 : time : 0:01:25.045002 - R^2 err : train[-1.6811281208228834e-05] test[-0.0012316338109319425]
5 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 2 - 100 : time : 0:02:36.775000 - R^2 err : train[-3.0200468050356477e-05] test[-0.0006302948192915547]
6 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 4 - 2 - 250 : time : 0:06:16.382002 - 

In [4]:
# model variables
iteration = '66'

flayers = [4,8,12,16]
slayers = [1,2,3,4,5,6]
epochs = [50,100,250,500]

loops = 0
hotStart = 95
for fullFileName in himawariData.keys():
    (trainData, trainLabel), (testData, testLabel) = himawariData[fullFileName]['traintest']
    for flayer in flayers:
        for slayer in slayers:
            for epoch in epochs:
                # hotstart
                if loops < hotStart:
                    loops+=1
                    continue
                
                tick = datetime.datetime.now()
                model = model6(flayer, slayer, epoch)

                # evaluating model accuracy
                prediction_model = tf.keras.Sequential([model,
                                                        tf.keras.layers.ReLU()])
                testPredictions = prediction_model.predict(testData)
                trainPredictions = prediction_model.predict(trainData)

                # make predictions
                testPredictions = testPredictions*(maxStat-minStat)+minStat
                trainPredictions = trainPredictions*(maxStat-minStat)+minStat
                realTestLabel = testLabel*(maxStat-minStat)+minStat
                realTrainLabel = trainLabel*(maxStat-minStat)+minStat

                # Mean Squared Error : 
                mse = tf.keras.losses.MeanSquaredError()
                mseTestError = mse(realTestLabel, testPredictions).numpy()
                mseTrainError = mse(realTrainLabel, trainPredictions).numpy()

                # RMSE
                rmseTest = rmse(np.squeeze(testPredictions), realTestLabel)
                rmseTrain = rmse(np.squeeze(trainPredictions), realTrainLabel)

                # NSE
                nseTest = nse(realTestLabel, np.squeeze(testPredictions))
                nseTrain = nse(realTrainLabel, np.squeeze(trainPredictions))

                # R^2
                r2Test = r2_score(realTestLabel, testPredictions)
                r2Train = r2_score(realTrainLabel, trainPredictions)

                # save statistics to csv
                statistics = '{},{},{},{},{},{},{},{},{},{},{}\n'.format(flayer, slayer, epoch, mseTrainError, mseTestError, rmseTrain, rmseTest, r2Train, r2Test, nseTrain, nseTest)
                with open('models_statistics/{}_GS_{}.csv'.format(iteration, himawariData[fullFileName]['filename']), 'a') as stat:
                    stat.write(statistics)

                # save model to drive
                model.save('models/{}/{}_GS_{}_{}_{}.h5'.format(iteration, himawariData[fullFileName]['filename'], flayer, slayer, epoch))

                # loop identifier :
                tock = datetime.datetime.now()
                print('{} : {} - {} - {} - {} : time : {} - R^2 err : train[{}] test[{}]'.format(loops, himawariData[fullFileName]['filename'], flayer, slayer, epoch, tock-tick, r2Train, r2Test))
                loops+=1
      

95 : cloudpsfqvaporrainsstwindwinuwn10784r1f - 16 - 6 - 500 : time : 0:05:02.498004 - R^2 err : train[-0.0002466770677496122] test[-0.00020968795797249307]
96 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 1 - 50 : time : 0:00:29.101016 - R^2 err : train[-5.127281041183984e-05] test[-0.0020755288450178266]
97 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 1 - 100 : time : 0:00:55.819984 - R^2 err : train[-0.0005108405815383854] test[-0.00024593019069296673]
98 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 1 - 250 : time : 0:02:16.648003 - R^2 err : train[-0.0006019349450925304] test[-0.00018881303984730202]
99 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 1 - 500 : time : 0:04:36.141000 - R^2 err : train[-0.0018116213377503687] test[-1.9006164491575106e-05]
100 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 2 - 50 : time : 0:00:28.285000 - R^2 err : train[-0.00035755288412886443] test[-0.003289327624359517]
101 : cloudpsfqvaporrainsstwindwinuwn10784r2f - 4 - 2 - 100 : time : 0:0

In [4]:
# load dataset
usedDatas = [[('cloud', 'psf', 'qvapor', 'rain', 'sst', 'wind', 'winu', 'wn10'),],[('psf','qvapor','rain','sst','wind','winu','wn10'),],[('cloud','psf','qvapor','rain','sst','wind','winu'),],[('cloud','psf','qvapor','rain','sst','wind','wn10'),],[('psf','qvapor','rain','sst','wind','winu'),]]
dims=([784,],[784,],[784,],[784,],[784,])
recurrentLists = (7,8,9,10,11,12,13,14,15,16,17)
WLSTATION = 'katulampa'
DATASCOPE = 'prediction'

# SLEEP #
#time.sleep(14400) # 4 hours waiting for manggarai RNN to be finished
#########

adte, adta = getAvailableSlicedData(dataScope=DATASCOPE, wlstation=WLSTATION)
adta = np.array(adta).astype('float32')
# normalizing input data
norm, minStat, maxStat = normalizingLabels(adta)

dataLabel = []
# create data Label for each recurrent number
for recurrentList in recurrentLists:
    recurrentIndexList, availableRecurrentDate, availableRecurrentLabel = generateRNNInput(adte, norm, recurrentCount=recurrentList)
    dataLabel.append(availableRecurrentLabel)
    
# DEFINE THIS FIRST #
usedDatas = usedDatas[0]
dims = dims[0]
# ################# #

himawariData={}
for i in range(len(usedDatas)):
    usedData = usedDatas[i]
    dim = dims[i]
    # define the length of data
    dataLength = len(usedData)
    # read stored data
    if dataLength == 1:
        fileName = '{}{}'.format(usedData[0], dim)
    elif dataLength == 2:
        fileName = '{}{}{}'.format(usedData[0], usedData[1], dim)
    elif dataLength == 3:
        fileName = '{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], dim)
    elif dataLength == 4:
        fileName = '{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], dim)
    elif dataLength == 5:
        fileName = '{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], dim)
    elif dataLength == 6:
        fileName = '{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], dim)
    elif dataLength == 7:
        fileName = '{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], dim)
    elif dataLength == 8:
        fileName = '{}{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], usedData[7], dim)
    print(fileName)
    
    for j in range(len(recurrentLists)):
        fullFileName = '{}r{}f'.format(fileName, recurrentLists[j])
        fpath = 'dataset/{}RNN/{}.hdf5'.format(WLSTATION, fullFileName)
        with h5py.File(fpath,'r') as f:
            data = f['datas'][()]
            
        print(data.shape, len(dataLabel[j]))

        himawariData[fullFileName]={'filename':'{}'.format(fullFileName),
                                    'traintest': splitTrainTest(data, dataLabel[j], split=0.7, shuffle=True, randomSeed=10)}


cloudpsfqvaporrainsstwindwinuwn10784
(2342, 50176) 2342
(1955, 56448) 1955
(1650, 62720) 1650
(1411, 68992) 1411
(1208, 75264) 1208
(1037, 81536) 1037
(896, 87808) 896
(778, 94080) 778
(675, 100352) 675
(591, 106624) 591
(516, 112896) 516


In [5]:
# model variables
iteration = '66'

flayers = [4,8,12,16]
slayers = [1,2,3,4,5,6]
epochs = [50,100,250,500]

loops = 0
hotStart = 75
for fullFileName in himawariData.keys():
    (trainData, trainLabel), (testData, testLabel) = himawariData[fullFileName]['traintest']
    for flayer in flayers:
        for slayer in slayers:
            for epoch in epochs:
                # hotstart
                if loops < hotStart:
                    loops+=1
                    continue
                
                tick = datetime.datetime.now()
                model = model6(flayer, slayer, epoch)

                # evaluating model accuracy
                prediction_model = tf.keras.Sequential([model,
                                                        tf.keras.layers.ReLU()])
                testPredictions = prediction_model.predict(testData)
                trainPredictions = prediction_model.predict(trainData)

                # make predictions
                testPredictions = testPredictions*(maxStat-minStat)+minStat
                trainPredictions = trainPredictions*(maxStat-minStat)+minStat
                realTestLabel = testLabel*(maxStat-minStat)+minStat
                realTrainLabel = trainLabel*(maxStat-minStat)+minStat

                # Mean Squared Error : 
                mse = tf.keras.losses.MeanSquaredError()
                mseTestError = mse(realTestLabel, testPredictions).numpy()
                mseTrainError = mse(realTrainLabel, trainPredictions).numpy()

                # RMSE
                rmseTest = rmse(np.squeeze(testPredictions), realTestLabel)
                rmseTrain = rmse(np.squeeze(trainPredictions), realTrainLabel)

                # NSE
                nseTest = nse(realTestLabel, np.squeeze(testPredictions))
                nseTrain = nse(realTrainLabel, np.squeeze(trainPredictions))

                # R^2
                r2Test = r2_score(realTestLabel, testPredictions)
                r2Train = r2_score(realTrainLabel, trainPredictions)

                # save statistics to csv
                statistics = '{},{},{},{},{},{},{},{},{},{},{}\n'.format(flayer, slayer, epoch, mseTrainError, mseTestError, rmseTrain, rmseTest, r2Train, r2Test, nseTrain, nseTest)
                with open('models_statistics/{}_GS_{}.csv'.format(iteration, himawariData[fullFileName]['filename']), 'a') as stat:
                    stat.write(statistics)

                # save model to drive
                model.save('models/{}/{}_GS_{}_{}_{}.h5'.format(iteration, himawariData[fullFileName]['filename'], flayer, slayer, epoch))

                # loop identifier :
                tock = datetime.datetime.now()
                print('{} : {} - {} - {} - {} : time : {} - R^2 err : train[{}] test[{}]'.format(loops, himawariData[fullFileName]['filename'], flayer, slayer, epoch, tock-tick, r2Train, r2Test))
                loops+=1
      

75 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 1 - 500 : time : 0:03:19.573312 - R^2 err : train[-0.0010980546267602698] test[-0.0006341909685225744]
76 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 2 - 50 : time : 0:00:20.871698 - R^2 err : train[-0.0029904772198396934] test[-0.012722476963546558]
77 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 2 - 100 : time : 0:00:40.867010 - R^2 err : train[-0.0002521843331508844] test[-0.0017976058657376282]
78 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 2 - 250 : time : 0:01:42.359898 - R^2 err : train[-2.193943348327565e-06] test[-0.003566760930461488]
79 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 2 - 500 : time : 0:03:36.569339 - R^2 err : train[-0.00030623044309985303] test[-0.0057301956970159384]
80 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 3 - 50 : time : 0:00:21.671100 - R^2 err : train[-1.1102392005746253e-07] test[-0.0034336737791873517]
81 : cloudpsfqvaporrainsstwindwinuwn10784r7f - 16 - 3 - 100 : time : 0:0

In [6]:
# load dataset
usedDatas = [[('cloud', 'psf', 'qvapor', 'rain', 'sst', 'wind', 'winu', 'wn10'),],[('psf','qvapor','rain','sst','wind','winu','wn10'),],[('cloud','psf','qvapor','rain','sst','wind','winu'),],[('cloud','psf','qvapor','rain','sst','wind','wn10'),],[('psf','qvapor','rain','sst','wind','winu'),]]
dims=([784,],[784,],[784,],[784,],[784,])
recurrentLists = (18,19,20,21,22,23,24)
WLSTATION = 'katulampa'
DATASCOPE = 'prediction'

# SLEEP #
#time.sleep(14400) # 4 hours waiting for manggarai RNN to be finished
#########

adte, adta = getAvailableSlicedData(dataScope=DATASCOPE, wlstation=WLSTATION)
adta = np.array(adta).astype('float32')
# normalizing input data
norm, minStat, maxStat = normalizingLabels(adta)

dataLabel = []
# create data Label for each recurrent number
for recurrentList in recurrentLists:
    recurrentIndexList, availableRecurrentDate, availableRecurrentLabel = generateRNNInput(adte, norm, recurrentCount=recurrentList)
    dataLabel.append(availableRecurrentLabel)
    
# DEFINE THIS FIRST #
usedDatas = usedDatas[0]
dims = dims[0]
# ################# #

himawariData={}
for i in range(len(usedDatas)):
    usedData = usedDatas[i]
    dim = dims[i]
    # define the length of data
    dataLength = len(usedData)
    # read stored data
    if dataLength == 1:
        fileName = '{}{}'.format(usedData[0], dim)
    elif dataLength == 2:
        fileName = '{}{}{}'.format(usedData[0], usedData[1], dim)
    elif dataLength == 3:
        fileName = '{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], dim)
    elif dataLength == 4:
        fileName = '{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], dim)
    elif dataLength == 5:
        fileName = '{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], dim)
    elif dataLength == 6:
        fileName = '{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], dim)
    elif dataLength == 7:
        fileName = '{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], dim)
    elif dataLength == 8:
        fileName = '{}{}{}{}{}{}{}{}{}'.format(usedData[0], usedData[1], usedData[2], usedData[3], usedData[4], usedData[5], usedData[6], usedData[7], dim)
    print(fileName)
    
    for j in range(len(recurrentLists)):
        fullFileName = '{}r{}f'.format(fileName, recurrentLists[j])
        fpath = 'dataset/{}RNN/{}.hdf5'.format(WLSTATION, fullFileName)
        with h5py.File(fpath,'r') as f:
            data = f['datas'][()]
            
        print(data.shape, len(dataLabel[j]))

        himawariData[fullFileName]={'filename':'{}'.format(fullFileName),
                                    'traintest': splitTrainTest(data, dataLabel[j], split=0.7, shuffle=True, randomSeed=10)}


cloudpsfqvaporrainsstwindwinuwn10784
(447, 119168) 447
(387, 125440) 387
(337, 131712) 337
(290, 137984) 290
(249, 144256) 249
(211, 150528) 211
(180, 156800) 180


In [7]:
# model variables
iteration = '66'

flayers = [4,8,12,16]
slayers = [1,2,3,4,5,6]
epochs = [50,100,250,500]

loops = 0
#hotStart = 75
for fullFileName in himawariData.keys():
    (trainData, trainLabel), (testData, testLabel) = himawariData[fullFileName]['traintest']
    for flayer in flayers:
        for slayer in slayers:
            for epoch in epochs:
                # hotstart
#                 if loops < hotStart:
#                     loops+=1
#                     continue
                
                tick = datetime.datetime.now()
                model = model6(flayer, slayer, epoch)

                # evaluating model accuracy
                prediction_model = tf.keras.Sequential([model,
                                                        tf.keras.layers.ReLU()])
                testPredictions = prediction_model.predict(testData)
                trainPredictions = prediction_model.predict(trainData)

                # make predictions
                testPredictions = testPredictions*(maxStat-minStat)+minStat
                trainPredictions = trainPredictions*(maxStat-minStat)+minStat
                realTestLabel = testLabel*(maxStat-minStat)+minStat
                realTrainLabel = trainLabel*(maxStat-minStat)+minStat

                # Mean Squared Error : 
                mse = tf.keras.losses.MeanSquaredError()
                mseTestError = mse(realTestLabel, testPredictions).numpy()
                mseTrainError = mse(realTrainLabel, trainPredictions).numpy()

                # RMSE
                rmseTest = rmse(np.squeeze(testPredictions), realTestLabel)
                rmseTrain = rmse(np.squeeze(trainPredictions), realTrainLabel)

                # NSE
                nseTest = nse(realTestLabel, np.squeeze(testPredictions))
                nseTrain = nse(realTrainLabel, np.squeeze(trainPredictions))

                # R^2
                r2Test = r2_score(realTestLabel, testPredictions)
                r2Train = r2_score(realTrainLabel, trainPredictions)

                # save statistics to csv
                statistics = '{},{},{},{},{},{},{},{},{},{},{}\n'.format(flayer, slayer, epoch, mseTrainError, mseTestError, rmseTrain, rmseTest, r2Train, r2Test, nseTrain, nseTest)
                with open('models_statistics/{}_GS_{}.csv'.format(iteration, himawariData[fullFileName]['filename']), 'a') as stat:
                    stat.write(statistics)

                # save model to drive
                model.save('models/{}/{}_GS_{}_{}_{}.h5'.format(iteration, himawariData[fullFileName]['filename'], flayer, slayer, epoch))

                # loop identifier :
                tock = datetime.datetime.now()
                print('{} : {} - {} - {} - {} : time : {} - R^2 err : train[{}] test[{}]'.format(loops, himawariData[fullFileName]['filename'], flayer, slayer, epoch, tock-tick, r2Train, r2Test))
                loops+=1
      

0 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 1 - 50 : time : 0:00:06.135996 - R^2 err : train[-0.032664691117915146] test[-0.004049660336118199]
1 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 1 - 100 : time : 0:00:11.731005 - R^2 err : train[-3.233278422776209e-08] test[-0.024834733118781482]
2 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 1 - 250 : time : 0:00:27.706995 - R^2 err : train[-1.6978458665661833e-06] test[-0.025278227177626622]
3 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 1 - 500 : time : 0:00:52.928999 - R^2 err : train[-0.0004271736689185346] test[-0.03337237089324452]
4 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 2 - 50 : time : 0:00:06.087003 - R^2 err : train[-0.006928739687590557] test[-0.003091499950051313]
5 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 2 - 100 : time : 0:00:11.039001 - R^2 err : train[-0.017132520715247956] test[-7.176828729971163e-06]
6 : cloudpsfqvaporrainsstwindwinuwn10784r18f - 4 - 2 - 250 : time : 0:00:26.718997 - R